In [ ]:
import os
import random
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split
from keras.utils import Sequence
import segmentation_models as sm
import tensorflow as tf
from keras import optimizers, models, callbacks

In [ ]:
%matplotlib inline
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"
plt.rcParams['figure.figsize'] = [12, 10]

_SEED = 42
random.seed(_SEED)
np.random.seed(_SEED)
# tf.compat.v1.random.set_random_seed(_SEED)

In [ ]:
data = pd.read_csv('../data/train.csv')
data.head()

In [ ]:
# Separate imageid
data['image_id'] = data.ImageId_ClassId.apply(lambda x: x.split('_')[0])
data['class_id'] = data.ImageId_ClassId.apply(lambda x: x.split('_')[1])
data.drop('ImageId_ClassId', axis=1, inplace=True)
data.head()

In [ ]:
train_df = pd.DataFrame({
    'image_id': data['image_id'][::4]
})
train_df['defect_1'] = data.EncodedPixels[::4].values
train_df['defect_2'] = data.EncodedPixels[1::4].values
train_df['defect_3'] = data.EncodedPixels[2::4].values
train_df['defect_4'] = data.EncodedPixels[3::4].values
train_df['defect_count'] = train_df[train_df.columns[1:]].count(axis=1)
train_df.reset_index(inplace=True, drop=True)
train_df.fillna('', inplace=True)
print(train_df.info())
train_df.head(10)

In [ ]:
# Utility functions for RL encoding/decoding
def mask2rle(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels= img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)
 
def rle2mask(mask_rle, shape=(1600,256)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (width,height) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T

In [ ]:
class DataSequence(Sequence):
    def __init__(self, df, batch_size, img_size,
                 base_path='../data/train_images',
                 train=True, n_classes=4, n_channels=3,
                 shuffle=False, augment=False):
        self.df = df
        self.batch_size = batch_size
        self.height, self.width = img_size
        self.base_path = base_path
        self.train = train
        self.n_classes = n_classes
        self.n_channels = n_channels
        self.shuffle = shuffle
        self.augment = augment
    
    def __len__(self):
        return int(np.ceil(len(self.df.index) / float(self.batch_size)))
    
    def __getitem__(self, idx):
        flip_direction = None
        batch = self.df[idx*self.batch_size: (idx+1)*self.batch_size].reset_index(drop=True)
        images = np.zeros((len(batch.index), self.height, self.width, self.n_channels))
        if self.train:
            masks = np.zeros((len(batch.index), self.height, self.width, self.n_classes), dtype='int')
        for row in batch.itertuples():
            image = cv2.imread(f'{self.base_path}/{row.image_id}')
            assert image.shape == (self.height, self.width, self.n_channels), f'Image shape not as expected, got {image.shape}, expected {(self.height, self.width, self.n_channels)}'
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            if self.augment:
                if random.random() > 0.5:
                    flip_direction = random.choice([-1, 0, 1])
                image = self.augment_image(image, row.defect_count, flip_direction=flip_direction)
            images[row.Index] = image / 255.
            if self.train:
                rles = row[2:-1]
                mask = self.build_mask(rles, flip_direction)
                masks[row.Index] = mask
        if self.train:
            return images, masks
        else:
            return images
    
    def on_epoch_end(self):
        if self.shuffle:
            self.df = self.df.sample(frac=1, random_state=_SEED).reset_index(drop=True)
    
    def illuminate(self, image):
        illumination = np.zeros((self.height, self.width), np.float32)
        cv2.circle(illumination, (random.choice(range(0, self.width)), random.choice(range(0, self.height))), random.choice(range(10, 150)), 1, -1, lineType=cv2.LINE_AA)
        illumination = cv2.GaussianBlur(illumination,(257,257),0)
        illumination = illumination.reshape(self.height, self.width, 1)
        image = image.astype(np.float32)/255
        image = image*(1+illumination*1.05)
        image = np.clip(image*255,0,255).astype(np.uint8)
        return image
    
    def augment_image(self, image, defect_count, flip_direction=None):
        if flip_direction is not None:
            image = cv2.flip(image, flip_direction)
        if random.random() > 0.5:
            alpha = random.uniform(1.0, 2.0) # Simple contrast control
            image = cv2.convertScaleAbs(image, alpha=alpha, beta=0)
        elif random.random() > 0.6:
            beta = random.randint(10, 61)    # Simple brightness control
            image = cv2.convertScaleAbs(image, alpha=1, beta=beta)
        if defect_count == 0 and False:
            if random.random() > 0.5:
                image = self.illuminate(image)
            if random.random() > 0.5:
                crop_hw = [random.choice(range(10, self.height//10)), random.choice(range(10, self.width//10))]
                image = image[crop_hw[0]:-crop_hw[0], crop_hw[1]:-crop_hw[1], :]
                image = cv2.resize(image, (self.width, self.height))
            if random.random() > 0.5:
                transformation_matrix = np.float32([[1,0,random.choice(range(-self.width//4, self.width//4))],[0,1,random.choice(range(-self.height//10, self.height//10))]])
                image = cv2.warpAffine(image, transformation_matrix, (self.width, self.height))
        return image
    
    def build_mask(self, rles, flip_direction=None):
        assert self.n_classes == len(rles), 'length of rles should be same as number of classes'
        mask = np.zeros((self.height, self.width, self.n_classes), dtype='int')
        for i, rle in enumerate(rles):
            if type(rle) is str:
                m = rle2mask(rle, (self.width, self.height))
                if flip_direction is not None:
                    mask[:, :, i] = cv2.flip(m, flip_direction)
                else:
                    mask[:, :, i] = m
        return mask

In [ ]:
def plot_image_with_mask(image, mask, title):
    img = image.copy()
    img[mask==1, 0] = 255
    plt.imshow(img)
    plt.title(title)
    plt.show()

In [ ]:
for x, y in DataSequence(train_df, 5, (256, 1600)):
    print(x.shape, y.shape)
    print(x.max(), x.min(), y.max(), y.min())
    for k in range(0, x.shape[0]):
        for i in range(0, y.shape[-1]):
            plot_image_with_mask(x[k], y[k][:,:,i], f'image {k}, defect {i}')
    break

In [ ]:
for x, y in DataSequence(train_df, 5, (256, 1600), augment=True):
    print(x.shape, y.shape)
    print(x.max(), x.min(), y.max(), y.min())
    for k in range(0, x.shape[0]):
        for i in range(0, y.shape[-1]):
            plot_image_with_mask(x[k], y[k][:,:,i], f'image {k}, defect {i}')
    break

In [ ]:
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=_SEED)
len(train_df.index), len(val_df.index)

In [ ]:
BACKBONE = 'resnet18'
BATCH_SIZE = 6
LR = 0.0001
CLASS_WEIGHTS = np.asarray([1.825, 1.952, 1, 1.844])

In [ ]:
train_seq = DataSequence(train_df, BATCH_SIZE, (256, 1600), shuffle=True, augment=True)
val_seq = DataSequence(val_df, BATCH_SIZE, (256, 1600), shuffle=False, augment=False)

In [ ]:
model = sm.Unet(BACKBONE, input_shape=(256, 1600, 3), classes=4, activation='sigmoid', encoder_weights='imagenet')

In [ ]:
model.compile(optimizers.Adam(lr=LR),
              sm.losses.bce_dice_loss,
              [sm.metrics.iou_score, sm.metrics.f1_score])

In [ ]:
reduce_lr = callbacks.ReduceLROnPlateau(patience=3, verbose=1)
early_stop = callbacks.EarlyStopping(patience=5, verbose=1, restore_best_weights=True)
ckpt = callbacks.ModelCheckpoint('models/weights-{epoch:04d}-{val_loss:.4f}.hdf5', verbose=1, save_best_only=True, save_weights_only=True)
training_callbacks = [reduce_lr, ckpt, early_stop]

In [ ]:
history = model.fit(train_seq, epochs=100, verbose=1, callbacks=training_callbacks, validation_data=val_seq, max_queue_size=4, workers=2, use_multiprocessing=True)

In [ ]:
model.save('models/final_model.hdf5')

In [ ]:
with open('history.json', 'w') as f:
    json.dump(str(history.history), f)

history_df = pd.DataFrame(history.history)
history_df[['loss', 'val_loss']].plot()
history_df[['iou_score', 'val_iou_score']].plot()
history_df[['score', 'val_score']].plot()